In [8]:
import pandas as pd
import chardet

with open(r"C:\Users\iftah\Desktop\data analyst studies\project\Ivry B\Leads_cleaning\4duplicates remove and merge with other source\csvleads_merged_uncleaned.csv", "rb") as f:
    result = chardet.detect(f.read())
    
faultyExcel = pd.read_csv(
    r"C:\Users\iftah\Desktop\data analyst studies\project\Ivry B\Leads_cleaning\4duplicates remove and merge with other source\csvleads_merged_uncleaned.csv", 
    encoding=result["encoding"]).fillna('none')
#observations with contact name are up-to-date organizatoina
faultyExcel.head()

,state,organization,site,contact name,function,email
0,alabama,alabama beekeepers association,https://www.alabamabeekeepers.com/,damon wallace damonwallace.beeman@gmail.com,president,damonwallace.beeman@gmail.com
1,alabama,blount county beekeepers association,http://www.k4vb.com/regional associations/blco...,harold deason,president,none
2,alabama,etowah county beekeepers association,http://etowahbeekeepers.org/,wil montgomery nwm@juno.com,president,nwm@juno.com
3,alabama,jefferson county bee keepers,none,danielle dowell,president,none
4,alabama,walker county beekeepers association,https://www.facebook.com/walker-county-beekeep...,president lonnie funderburg funter@otelco.net,president,funter@otelco.net


In [9]:
faultyExcel = faultyExcel.apply(lambda x: x.str.strip().str.lower())
faultyExcelBACKUP = pd.DataFrame(faultyExcel)
duplicates = faultyExcel.loc[faultyExcel['organization'].duplicated(keep=False)].sort_values('organization')
                        
#partly missing information in state for instance: north_ 

duplicate_2_words_miss = duplicates.loc[duplicates['state'].str.rfind(' ') == -1].sort_values('state')
# new, north, south are first words of missing data

unmergedTwins = duplicates.loc[duplicates['state'].str.contains('new|north|south')]
#by looking at the duplicates we can understand to which one it belongs and consider dropping it
unmergedTwins.sort_values('organization').head(10)

,state,organization,site,contact name,function,email
465,south carolina,aiken county beekeepers association,http://aikenbeekeepers.org/,none,none,none
905,south carolina,aiken county beekeepers association,www.aikenbeekeepers.org,robert abershire,president,abshirefamily4@bellsouth.net
907,south carolina,anderson county beekeepers association,https://www.andersonbeekeepers.org,tony olekas,president,acbaofsc@gmail.com
466,south carolina,anderson county beekeepers association,http://www.andersonbeekeepers.org/,none,none,none
68,north,beekeepers of wilkes county,none,jon molesa ..,president,president@beekeepersofwilkes.org
359,north carolina,beekeepers of wilkes county,http://www.beekeepersofwilkes.org/,none,none,none
467,south carolina,blackwater beekeepers association,http://www.blackwaterbeekeepers.org/,none,none,none
909,south carolina,blackwater beekeepers association,www.blackwaterbeekeepers.org,keith martin,president,ionlydriveaford@yahoo.com
318,new hampshire,capital area beekeepers association,http://www.capitalareabeekeepers.org/,none,none,none
336,new york,catskill mountain beekeepers club,http://www.catskillbees.org/,none,none,none


In [10]:
def select_longer_text(x):
    try:
        if len(x.iloc[0]) >= len(x.iloc[1]):
            return x.iloc[0]
        else:
            return x.iloc[1]
    except IndexError:
            return x

In [15]:
mergedTwins = unmergedTwins.groupby('organization', as_index=False).\
agg({'state':select_longer_text, 'contact name':select_longer_text,\
                                                                    'email':select_longer_text, 'site':select_longer_text,\
                                                                    'function':select_longer_text}).sort_values('organization')
mergedTwins.head(10)

,organization,state,contact name,email,site,function
0,aiken county beekeepers association,south carolina,robert abershire,abshirefamily4@bellsouth.net,http://aikenbeekeepers.org/,president
1,anderson county beekeepers association,south carolina,tony olekas,acbaofsc@gmail.com,https://www.andersonbeekeepers.org,president
2,beekeepers of wilkes county,north carolina,jon molesa ..,president@beekeepersofwilkes.org,http://www.beekeepersofwilkes.org/,president
3,blackwater beekeepers association,south carolina,keith martin,ionlydriveaford@yahoo.com,http://www.blackwaterbeekeepers.org/,president
4,capital area beekeepers association,new hampshire,none,none,http://www.capitalareabeekeepers.org/,none
5,catskill mountain beekeepers club,new york,richard j.,ronconi rronconi@att.net,http://www.catskillbees.org/,president
6,chester county beekeepers association,south carolina,alyson wissinger,chestercountybeekeepers@gmail.com,https://chestercountybeekeepers.org,president
7,franklin county beekeepers association,north carolina,none,none,http://www.franklincountybees.org/,none
8,guilford county beekeepers,north carolina,robert j. jacobs robertjjacobs@aol.com,robertjjacobs@aol.com,http://guilfordbeekeepers.org/,president
9,henderson county beekeepers association,north carolina,none,â info@hcbeekeepers.org,http://hcbeekeepers.org/,none


In [12]:
#now we'll merge it back to the original dataframe, and remove the former obesarvations
faultyExcel.index = list(faultyExcel.index)
faultyExcel.drop(index=unmergedTwins.index, inplace=True)

In [14]:
mergedTwins = mergedTwins[['state', 'organization', 'site', 'contact name','function', 'email']]
leads_final = faultyExcel.append(mergedTwins)


C:\Users\iftah\AppData\Local\Temp\ipykernel_2596\2024453131.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  leads_final = faultyExcel.append(mergedTwins)
